In [1]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('train', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading train -------
Read 45544 sentence pairs
Counting words
Counted words:
In questions: 16871 words
In answers: 14637 words


In [4]:
# Add EOS at end of each target sentence
pairs = [[line[0], line[1]+' EOS'] for line in pairs ]

In [5]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [6]:
criterion = nn.NLLLoss()
optimizer = 'Adam'
batch_size = 32
lr = 0.0005

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.2, 'general', device, criterion)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [9]:
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, threshold=0.001, min_lr=1e-6)
#scheduler = CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=500)

In [10]:
a.n_words

14637

In [11]:
from jupyterplot import ProgressPlot

pp = ProgressPlot(x_label='Epoch', plot_names=['train_loss', 'train_accuracy'])

for epoch in tnrange(1000, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')

    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, a, device)
    
    #current_lr = optimizer.param_groups[0]['lr']
    
    print(f'Loss: {loss}')
    #print(f'LR: {current_lr}')
    
    #scheduler.step(loss)
    #scheduler.step()

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, q, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    #BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    #bleu_score = BLEU_model.score()

    #print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    pp.update([[loss], [accuracy]])

pp.finalize()
print(f"Optimization ended successfully")


Total epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 2021-09-24 18:35:05.856454
Loss: 3.742639730646694
Train accuracy: 0.021204686984133806
Epoch 2: 2021-09-24 18:38:26.638701
Loss: 3.272500628002866
Train accuracy: 0.017963362028818323
Epoch 3: 2021-09-24 18:41:45.943423
Loss: 3.112162533976142
Train accuracy: 0.03143104923990297
Epoch 4: 2021-09-24 18:45:04.663025
Loss: 3.0542262469585397
Train accuracy: 0.03374189519554322
Epoch 5: 2021-09-24 18:48:22.770183
Loss: 3.012576968624181
Train accuracy: 0.031904219650822185
Epoch 6: 2021-09-24 18:51:41.291349
Loss: 2.9450702418280152
Train accuracy: 0.05986062671323666
Epoch 7: 2021-09-24 18:54:59.450064
Loss: 2.933636125811017
Train accuracy: 0.05371417529302099
Epoch 8: 2021-09-24 18:58:17.884335
Loss: 2.9146145182465575
Train accuracy: 0.060943466379558664
Epoch 9: 2021-09-24 19:01:35.662378
Loss: 2.890132537144159
Train accuracy: 0.05001020373603003
Epoch 10: 2021-09-24 19:04:52.668418
Loss: 2.843672557445267
Train accuracy: 0.06780307110713932
Epoch 11: 2021-09-24 19:08:09.84

Loss: 2.2080937808611543
Train accuracy: 0.13193973638281026
Epoch 86: 2021-09-24 23:16:19.976758
Loss: 2.2126829656675504
Train accuracy: 0.13488835085291187
Epoch 87: 2021-09-24 23:19:38.905477
Loss: 2.208602708454467
Train accuracy: 0.13459694075633966
Epoch 88: 2021-09-24 23:22:57.819740
Loss: 2.2096768024075053
Train accuracy: 0.13259572760238458
Epoch 89: 2021-09-24 23:26:16.276766
Loss: 2.163360904105286
Train accuracy: 0.14399533036757914
Epoch 90: 2021-09-24 23:29:35.816164
Loss: 2.206335339521987
Train accuracy: 0.13593046431838432
Epoch 91: 2021-09-24 23:32:54.951534
Loss: 2.209363060298062
Train accuracy: 0.13874028338451969
Epoch 92: 2021-09-24 23:36:14.380334
Loss: 2.1775993703929384
Train accuracy: 0.14761827073978007
Epoch 93: 2021-09-24 23:39:33.405768
Loss: 2.1805601112882362
Train accuracy: 0.13212809025069824
Epoch 94: 2021-09-24 23:42:51.934857
Loss: 2.1670949572957436
Train accuracy: 0.1474423836453676
Epoch 95: 2021-09-24 23:46:11.082814
Loss: 2.148443710163543
T

Epoch 169: 2021-09-25 03:51:50.838935
Loss: 2.030640858762233
Train accuracy: 0.17472572966891514
Epoch 170: 2021-09-25 03:55:09.159764
Loss: 1.963953228870723
Train accuracy: 0.18685168101600763
Epoch 171: 2021-09-25 03:58:29.149226
Loss: 2.011835857853396
Train accuracy: 0.17820497356298798
Epoch 172: 2021-09-25 04:01:47.970329
Loss: 2.0098774624763363
Train accuracy: 0.17544305293259707
Epoch 173: 2021-09-25 04:05:06.928089
Loss: 2.0111673110211723
Train accuracy: 0.1777913595875301
Epoch 174: 2021-09-25 04:08:25.744608
Loss: 1.9902232364497416
Train accuracy: 0.17712252885039195
Epoch 175: 2021-09-25 04:11:45.360649
Loss: 2.0529769452529374
Train accuracy: 0.17846355990945412
Epoch 176: 2021-09-25 04:15:05.042348
Loss: 1.990149116152831
Train accuracy: 0.17875182113341198
Epoch 177: 2021-09-25 04:18:23.675676
Loss: 1.9833537905684433
Train accuracy: 0.18147627216931497
Epoch 178: 2021-09-25 04:21:42.874314
Loss: 1.986429479540399
Train accuracy: 0.17497954195809548
Epoch 179: 2021-

Loss: 1.9249887495675087
Train accuracy: 0.20488849974654744
Epoch 253: 2021-09-25 08:30:35.073162
Loss: 1.905191537053602
Train accuracy: 0.20020107386544717
Epoch 254: 2021-09-25 08:33:53.141913
Loss: 1.8762987936049462
Train accuracy: 0.20308866961860036
Epoch 255: 2021-09-25 08:37:11.985611
Loss: 1.9233617485098156
Train accuracy: 0.20338646186226916
Epoch 256: 2021-09-25 08:40:30.917263
Loss: 1.9140923265981638
Train accuracy: 0.2000410734981638
Epoch 257: 2021-09-25 08:43:50.322090
Loss: 1.8814057277148566
Train accuracy: 0.20406150291584976
Epoch 258: 2021-09-25 08:47:09.814239
Loss: 1.87059615383183
Train accuracy: 0.20123777154179312
Epoch 259: 2021-09-25 08:50:28.596322
Loss: 1.912765600897953
Train accuracy: 0.20581005124351867
Epoch 260: 2021-09-25 08:53:46.726236
Loss: 1.8961850089679912
Train accuracy: 0.20504624422493536
Epoch 261: 2021-09-25 08:57:06.227005
Loss: 1.8992725018430932
Train accuracy: 0.1985090625578053
Epoch 262: 2021-09-25 09:00:25.122956
Loss: 1.87890139

...

Epoch 586: 2021-09-26 02:46:07.666308
Loss: 1.6794126170018286
Train accuracy: 0.25840184713536724
Epoch 587: 2021-09-26 02:49:22.795121
Loss: 1.7337743148647689
Train accuracy: 0.25740179662988893
Epoch 588: 2021-09-26 02:52:38.094286
Loss: 1.7058399558947293
Train accuracy: 0.25736085065170267
Epoch 589: 2021-09-26 02:55:52.534938
Loss: 1.7139709481795287
Train accuracy: 0.2565698656806522
Epoch 590: 2021-09-26 02:59:08.420139
Loss: 1.7152940937749157
Train accuracy: 0.2567476149797442
Epoch 591: 2021-09-26 03:02:23.495944
Loss: 1.691444680016356
Train accuracy: 0.2597220421967151
Epoch 592: 2021-09-26 03:05:38.300150
Loss: 1.683917872517784
Train accuracy: 0.259128659152536
Epoch 593: 2021-09-26 03:08:53.076514
Loss: 1.7158908201253993
Train accuracy: 0.2552907550424376
Epoch 594: 2021-09-26 03:12:08.484500
Loss: 1.7206780665380181
Train accuracy: 0.25926022776966995
Epoch 595: 2021-09-26 03:15:23.476522
Loss: 1.6998647463068506
Train accuracy: 0.25970196239242305
Epoch 596: 2021-09

Train accuracy: 0.26263678443928723
Epoch 670: 2021-09-26 07:19:15.313846
Loss: 1.707104108563428
Train accuracy: 0.26309695653837345
Epoch 671: 2021-09-26 07:22:30.642145
Loss: 1.7190382306459946
Train accuracy: 0.26180551128249335
Epoch 672: 2021-09-26 07:25:45.159500
Loss: 1.6760528157065913
Train accuracy: 0.2670299886333844
Epoch 673: 2021-09-26 07:29:06.703179
Loss: 1.6724176960513764
Train accuracy: 0.26505586863854125
Epoch 674: 2021-09-26 07:32:24.144790
Loss: 1.6627091252985915
Train accuracy: 0.26240002523390177
Epoch 675: 2021-09-26 07:35:40.169457
Loss: 1.6650877379237703
Train accuracy: 0.2660117624051105
Epoch 676: 2021-09-26 07:38:55.633326
Loss: 1.716086784152585
Train accuracy: 0.2679898705517343
Epoch 677: 2021-09-26 07:42:11.111024
Loss: 1.740360867093052
Train accuracy: 0.26495097674346174
Epoch 678: 2021-09-26 07:45:26.998411
Loss: 1.6565292058280283
Train accuracy: 0.26219709133540414
Epoch 679: 2021-09-26 07:48:42.792213
Loss: 1.6766386756688498
Train accuracy: 

Loss: 1.6898689118587673
Train accuracy: 0.2701471494692928
Epoch 754: 2021-09-26 11:53:17.198423
Loss: 1.6731450225621516
Train accuracy: 0.27024303370136005
Epoch 755: 2021-09-26 11:56:33.932989
Loss: 1.7145873362362185
Train accuracy: 0.2743450448491081
Epoch 756: 2021-09-26 11:59:49.611149
Loss: 1.6826036929552308
Train accuracy: 0.2732065420855162
Epoch 757: 2021-09-26 12:03:05.995263
Loss: 1.712239855279273
Train accuracy: 0.2730906091688332
Epoch 758: 2021-09-26 12:06:22.339820
Loss: 1.6417371695501395
Train accuracy: 0.2671902984130117
Epoch 759: 2021-09-26 12:09:37.634228
Loss: 1.6670234243398934
Train accuracy: 0.2821885649948787
Epoch 760: 2021-09-26 12:12:53.759786
Loss: 1.6924961076313414
Train accuracy: 0.28015982675681
Epoch 761: 2021-09-26 12:16:09.618193
Loss: 1.6599677136599553
Train accuracy: 0.2795584849394644
Epoch 762: 2021-09-26 12:19:24.783632
Loss: 1.6697741146454488
Train accuracy: 0.26858819601287626
Epoch 763: 2021-09-26 12:22:39.497847
Loss: 1.6936533586540

Epoch 837: 2021-09-26 16:23:36.840126
Loss: 1.6221989349611206
Train accuracy: 0.27259640005511837
Epoch 838: 2021-09-26 16:26:52.073482
Loss: 1.677772254870183
Train accuracy: 0.2798186937561066
Epoch 839: 2021-09-26 16:30:07.427951
Loss: 1.6645512543246488
Train accuracy: 0.27829566218681534
Epoch 840: 2021-09-26 16:33:23.294090
Loss: 1.6526919441138745
Train accuracy: 0.2760730658821484
Epoch 841: 2021-09-26 16:36:38.600182
Loss: 1.6674332128641705
Train accuracy: 0.2751888237941778
Epoch 842: 2021-09-26 16:39:54.337711
Loss: 1.690913697030784
Train accuracy: 0.27684435846711974
Epoch 843: 2021-09-26 16:43:09.782808
Loss: 1.6348455354833797
Train accuracy: 0.2758945273420665
Epoch 844: 2021-09-26 16:46:25.261054
Loss: 1.658818076626793
Train accuracy: 0.2741914053676721
Epoch 845: 2021-09-26 16:49:40.745048
Loss: 1.648163070761696
Train accuracy: 0.272777635322236
Epoch 846: 2021-09-26 16:52:55.393005
Loss: 1.6775558551669623
Train accuracy: 0.27360943132263676
Epoch 847: 2021-09-26

Epoch 921: 2021-09-26 20:57:09.142884
Loss: 1.6376697486312408
Train accuracy: 0.28248744349323074
Epoch 922: 2021-09-26 21:00:25.026499
Loss: 1.6770233742370924
Train accuracy: 0.27931438418489407
Epoch 923: 2021-09-26 21:03:39.847699
Loss: 1.691404560142255
Train accuracy: 0.2802367784506096
Epoch 924: 2021-09-26 21:06:54.319675
Loss: 1.6998984457033275
Train accuracy: 0.279399253259563
Epoch 925: 2021-09-26 21:10:09.189968
Loss: 1.6589709137429145
Train accuracy: 0.2832752997767353
Epoch 926: 2021-09-26 21:13:23.943316
Loss: 1.6450149614523808
Train accuracy: 0.2779154429873467
Epoch 927: 2021-09-26 21:16:39.620575
Loss: 1.7054010218029092
Train accuracy: 0.28591442486299024
Epoch 928: 2021-09-26 21:19:54.178399
Loss: 1.672830812421705
Train accuracy: 0.2828965759472022
Epoch 929: 2021-09-26 21:23:08.974326
Loss: 1.6276371655119557
Train accuracy: 0.27709624167199526
Epoch 930: 2021-09-26 21:26:23.908753
Loss: 1.6537806738003678
Train accuracy: 0.2824327382225444
Epoch 931: 2021-09-